In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# NVTabular / HugeCTR Criteo Example 
Here we'll show how to use NVTabular first as a preprocessing library to prepare the [Criteo Display Advertising Challenge](https://www.kaggle.com/c/criteo-display-ad-challenge) dataset, and then train a model using HugeCTR.

### Data Prep
Before we get started, make sure you've run the [optimize_criteo notebook](../optimize_criteo.ipynb), which will convert the tsv data published by Criteo into the parquet format that our accelerated readers prefer. It's fair to mention at this point that that notebook will take ~30 minutes to run. While we're hoping to release accelerated csv readers in the near future, we also believe that inefficiencies in existing data representations like csv are in no small part a consequence of inefficiencies in the existing hardware/software stack. Accelerating these pipelines on new hardware like GPUs may require us to make new choices about the representations we use to store that data, and parquet represents a strong alternative.

In [2]:
# Standard Libraries
import os
from time import time
import re
import shutil
import glob
import warnings

# External Dependencies
import cupy as cp
import cudf
import dask_cudf
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
from dask.utils import parse_bytes
from dask.delayed import delayed
import rmm

# NVTabular
import nvtabular as nvt
import nvtabular.ops as ops
from nvtabular.io import Shuffle
from nvtabular.utils import device_mem_size


### Dataset and Dataset Schema
Once our data is ready, we'll define some high level parameters to describe where our data is and what it "looks like" at a high level.

In [3]:
# define some information about where to get our data
BASE_DIR = "/raid/criteo/tests/"
input_path = os.path.join(BASE_DIR, "crit_int_pq")
dask_workdir = os.path.join(BASE_DIR, "test_dask/workdir")
output_path = os.path.join(BASE_DIR, "test_dask/output")
stats_path = os.path.join(BASE_DIR, "test_dask/stats")


BATCH_SIZE = int(os.environ.get('BATCH_SIZE', 800000))
NUM_PARTS = int(os.environ.get('NUM_PARTS', 2))
NUM_TRAIN_DAYS = 23 # number of days worth of data to use for training, the rest will be used for validation

# define our dataset schema
CONTINUOUS_COLUMNS = ['I' + str(x) for x in range(1,14)]
CATEGORICAL_COLUMNS =  ['C' + str(x) for x in range(1,27)]
LABEL_COLUMNS = ['label']
COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS + LABEL_COLUMNS

# Get device memory capacity
capacity = device_mem_size(kind="total")

# Make sure we have a clean worker space for Dask
if os.path.isdir(dask_workdir):
    shutil.rmtree(dask_workdir)
os.makedirs(dask_workdir)

# Make sure we have a clean stats space for Dask
if os.path.isdir(stats_path):
    shutil.rmtree(stats_path)
os.mkdir(stats_path)
         
# Make sure we have a clean output path
if os.path.isdir(output_path):
    shutil.rmtree(output_path)
os.mkdir(output_path)

In [4]:
! ls $BASE_DIR

crit_int_pq  criteo-hugectr.ipynb  hugectrchecker.cpp  numreader.py  test_dask


In [5]:
fname = 'day_{}.parquet'
num_days = len([i for i in os.listdir(input_path) if re.match(fname.format('[0-9]{1,2}'), i) is not None])
train_paths = [os.path.join(input_path, fname.format(day)) for day in range(NUM_TRAIN_DAYS)]
valid_paths = [os.path.join(input_path, fname.format(day)) for day in range(NUM_TRAIN_DAYS, num_days)]
print(train_paths)
print(valid_paths)

['/raid/criteo/tests/crit_int_pq/day_0.parquet', '/raid/criteo/tests/crit_int_pq/day_1.parquet', '/raid/criteo/tests/crit_int_pq/day_2.parquet', '/raid/criteo/tests/crit_int_pq/day_3.parquet', '/raid/criteo/tests/crit_int_pq/day_4.parquet', '/raid/criteo/tests/crit_int_pq/day_5.parquet', '/raid/criteo/tests/crit_int_pq/day_6.parquet', '/raid/criteo/tests/crit_int_pq/day_7.parquet', '/raid/criteo/tests/crit_int_pq/day_8.parquet', '/raid/criteo/tests/crit_int_pq/day_9.parquet', '/raid/criteo/tests/crit_int_pq/day_10.parquet', '/raid/criteo/tests/crit_int_pq/day_11.parquet', '/raid/criteo/tests/crit_int_pq/day_12.parquet', '/raid/criteo/tests/crit_int_pq/day_13.parquet', '/raid/criteo/tests/crit_int_pq/day_14.parquet', '/raid/criteo/tests/crit_int_pq/day_15.parquet', '/raid/criteo/tests/crit_int_pq/day_16.parquet', '/raid/criteo/tests/crit_int_pq/day_17.parquet', '/raid/criteo/tests/crit_int_pq/day_18.parquet', '/raid/criteo/tests/crit_int_pq/day_19.parquet', '/raid/criteo/tests/crit_int_

### Deploy a Distributed-Dask Cluster

In [7]:
# Config for 8xV100-32GB
# Deploy a Single-Machine Multi-GPU Cluster
protocol = "tcp"             # "tcp" or "ucx"
visible_devices = "0,1,2,3,4,5,6,7"  # Delect devices to place workers
device_limit_frac = 0.8      # Spill GPU-Worker memory to host at this limit.
device_pool_frac = 0.9
partition_size = 0.12
cluster = None               # (Optional) Specify existing scheduler port
if cluster is None:
    cluster = LocalCUDACluster(
        protocol = protocol,
        n_workers=len(visible_devices.split(",")),
        CUDA_VISIBLE_DEVICES = visible_devices,
        local_directory = dask_workdir,
        device_memory_limit = capacity * device_limit_frac,
    )

# Create the distributed client
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:36751 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 135.11 GB


#### Initilize Memory Pools

In [8]:
# Initialize RMM pool on ALL workers
def _rmm_pool():
    rmm.reinitialize(
        pool_allocator=True,
        initial_pool_size=(capacity * device_pool_frac), # Use default size
    )
    
client.run(_rmm_pool)

{'tcp://127.0.0.1:39147': None,
 'tcp://127.0.0.1:40115': None,
 'tcp://127.0.0.1:40289': None,
 'tcp://127.0.0.1:40921': None}

### Preprocessing
At this point, our data still isn't in a form that's ideal for consumption by neural networks. The most pressing issues are missing values and the fact that our categorical variables are still represented by random, discrete identifiers, and need to be transformed into contiguous indices that can be leveraged by a learned embedding. Less pressing, but still important for learning dynamics, are the distributions of our continuous variables, which are distributed across multiple orders of magnitude and are uncentered (i.e. E[x] != 0).

We can fix these issues in a conscise and GPU-accelerated manner with an NVTabular `Workflow`. We'll instantiate one with our current dataset schema, then symbolically add operations _on_ that schema. By setting all these `Ops` to use `replace=True`, the schema itself will remain unmodified, while the variables represented by each field in the schema will be transformed.

#### Frequency Thresholding
One interesting thing worth pointing out is that we're using _frequency thresholding_ in our `Categorify` op. This handy functionality will map all categories which occur in the dataset with some threshold level of infrequency (which we've set here to be 15 occurrences throughout the dataset) to the _same_ index, keeping the model from overfitting to sparse signals.

In [9]:
proc = nvt.Workflow(
    cat_names=CATEGORICAL_COLUMNS,
    cont_names=CONTINUOUS_COLUMNS,
    label_name=LABEL_COLUMNS,
    client = client)

# log -> normalize continuous features. Note that doing this in the opposite
# order wouldn't make sense! Note also that we're zero filling continuous
# values before the log: this is a good time to remember that LogOp
# performs log(1+x), not log(x)
proc.add_cont_feature([ops.FillMissing(), ops.Clip(min_value=0), ops.LogOp()])
proc.add_cont_preprocess(ops.Normalize())

# MOD 10M
proc.add_cat_preprocess(ops.HashBucket(10000000))
# categorification
proc.add_cat_preprocess(ops.Categorify(dtype = np.int64, out_path=OUTPUT_DATA_DIR))

Now instantiate dataset iterators to loop through our dataset (which we couldn't fit into GPU memory). We need to enforce the required HugeCTR data types, so we set them in a dictionary and give as an argument when creating our dataset

In [10]:
import numpy as np

dict_dtypes={}

for col in CONTINUOUS_COLUMNS:
    dict_dtypes[col] = np.float32
 
# Categorical output datatypes are specified in categorify operation
# for col in CATEGORICAL_COLUMNS:
#    dict_dtypes[col] = np.int64
    
for col in LABEL_COLUMNS:
    dict_dtypes[col] = np.float32

In [11]:
train_dataset = nvt.Dataset(train_paths, engine='parquet', part_size=capacity * partition_size, dtypes=dict_dtypes)
valid_dataset = nvt.Dataset(valid_paths, engine='parquet', part_size=capacity * partition_size, dtypes=dict_dtypes)

Now run them through our workflows to collect statistics on the train set, then transform and save to parquet files.

In [12]:
output_train_dir = os.path.join(output_path, 'train/')
output_valid_dir = os.path.join(output_path, 'valid/')
! mkdir -p $output_train_dir
! mkdir -p $output_valid_dir

For reference, let's time it to see how long it takes...

In [ ]:
%%time
proc.apply(train_dataset, shuffle=nvt.io.Shuffle.PER_PARTITION, output_format="parquet", output_path=output_train_dir)

In [ ]:
%%time
proc.apply(valid_dataset, record_stats=False, shuffle=nvt.io.Shuffle.PER_PARTITION, output_format="parquet", output_path=output_valid_dir)

In [ ]:
embeddings = ops.get_embedding_sizes(proc)
print(embeddings.values())

And just like that, we have training and validation sets ready to feed to a model!

## HugeCTR
### Training
We'll run huge_ctr using the DLRM configuration file.

First, we'll reinitialize our memory pool from earlier to free up some memory so that we can share it with HugeCTR.

In [14]:
rmm.reinitialize(pool_allocator=False)

Finally, we run HugeCTR. For reference, let's time it to see how long it takes...

In [27]:
%%time
os.system("/usr/local/hugectr/bin/huge_ctr --train dlrm_fp32_16k.json")

[0, init_start, ]
HugeCTR Version: 2.2.1
Config file: dlrm_fp32_16k.json
[28d23h07m10s][HUGECTR][INFO]: algorithm_search is not specified using default: 1
[28d23h07m10s][HUGECTR][INFO]: Algorithm search: ON
Device 0: Tesla P100-DGXS-16GB
Device 1: Tesla P100-DGXS-16GB
[28d23h07m15s][HUGECTR][INFO]: Initial seed is 104555387
[28d23h07m15s][HUGECTR][INFO]: cache_eval_data is not specified using default: 0
[28d23h07m15s][HUGECTR][INFO]: num_internal_buffers 1
[28d23h07m15s][HUGECTR][INFO]: num_internal_buffers 1
[28d23h07m15s][HUGECTR][INFO]: Vocabulary size: 31457706
[28d23h07m15s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=15500000
[28d23h07m15s][HUGECTR][INFO]: All2All Warmup Start
[28d23h07m15s][HUGECTR][INFO]: All2All Warmup End
[28d23h07m36s][HUGECTR][INFO]: gpu0 start to init embedding
[28d23h07m36s][HUGECTR][INFO]: gpu1 start to init embedding
[28d23h07m37s][HUGECTR][INFO]: gpu0 init embedding done
[28d23h07m37s][HUGECTR][INFO]: gpu1 init embedding done
[28418.6, init_end, ]
[28